In [1]:
from trainingLib import *
from collections import defaultdict

W0925 11:54:24.364768 26320 deprecation_wrapper.py:119] From C:\Users\Zifeng Li\Desktop\HelmsleyDocker\Helmsley3DMIP\_Scripts\Utilities\networkTrain\test_model.py:4: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W0925 11:54:24.366768 26320 deprecation_wrapper.py:119] From C:\Users\Zifeng Li\Desktop\HelmsleyDocker\Helmsley3DMIP\_Scripts\Utilities\networkTrain\test_model.py:4: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



In [2]:
# Run this block to prepare all the training data
pruneSystem()


Deleting old training files and temp files.
Deleting directory ./sampledDataset failed
Deleting directory ./sampledLabel failed


In [3]:
def getDataPaths():

    dicomDataPaths = defaultdict(list)
    maskDataPaths = defaultdict(list)

    for root, dirs, files in os.walk(".\Dataset\\training", topdown=True):

        for name in files:
            pre, dataId = path.split(root)
            pre, category = path.split(pre)

            if(category == 'image'):
                dicomDataPaths[dataId].append(path.join(root, name))
            elif(category == 'mask'):
                maskDataPaths[dataId].append(path.join(root, name))
    
    return dicomDataPaths, maskDataPaths

In [4]:
def getDataArray(dicomPaths:dict, maskPaths:dict, sortingDirection:int):
    
    if(set(dicomPaths.keys()) != set(maskPaths.keys())):
        print("Error! The series IDs between dicom paths and mask paths don't match.")
        exit()
    
    dicomVolumes = defaultdict(list)
    maskVolumes = defaultdict(list)
    
    for dataId in dicomPaths:
        # Sort all the dicom fules according to the patient position and store them in an array
        image_dcms = [pydicom.read_file(f, force=True) for f in dicomPaths[dataId] if f.endswith(".dcm")]
        image_dcms.sort(key = lambda x: int(x[0x20, 0x32][sortingDirection]))
        
        # Sort all the pngs according their names and store them in an array
        png_path_list = maskPaths[dataId]
        path_list_pre_parsed = [path.split(p) for p in png_path_list]
        path_list_parsed = [ [p[0]] + p[1].split(".") for p in path_list_pre_parsed]
        path_list_parsed_valid = [x for x in path_list_parsed if x[-1] == 'png']
        path_list_parsed_sorted = sorted(path_list_parsed_valid, key=lambda x:int(x[-2]))
        path_list_pre_joined = [ [p[0]] + [(p[1] + (".png"))] for p in path_list_parsed_sorted]
        path_list_joined = [path.join(p[0], p[1]) for p in path_list_pre_joined]
        mask_pngs = [np.array(Image.open(s)) for s in path_list_joined]
        
        if(len(image_dcms) != len(mask_pngs)):
            print("Error! The number of slices of the image dicoms does not agree with that of the mask dicoms. Abort!")
            exit()
            
        dicomVolumes[dataId] = image_dcms
        maskVolumes[dataId] = mask_pngs
    
    return dicomVolumes, maskVolumes

In [5]:
def createTrainingData(dicomPaths:dict, maskPaths:dict, sortingDirection:int):
    print("Creating training PNG files.")
    systemTempPath = "./tmp"
    
    try:
        os.mkdir(systemTempPath)
    except OSError:
        print ("Creation of the directory %s failed" % systemTempPath)
    else:
        print ("Successfully created the directory %s " % systemTempPath)

    trainingImagePath = "./tmp/trainingImage"
    
    try:
        os.mkdir(trainingImagePath)
    except OSError:
        print ("Creation of the directory %s failed" % trainingImagePath)
    else:
        print ("Successfully created the directory %s " % trainingImagePath)
        
    trainingMaskPath = "./tmp/trainingMask"
    
    try:
        os.mkdir(trainingMaskPath)
    except OSError:
        print ("Creation of the directory %s failed" % trainingMaskPath)
    else:
        print ("Successfully created the directory %s " % trainingMaskPath)
    
    dicomVol, maskVol = getDataArray(dicomPaths, maskPaths, sortingDirection)
    slice_count = 0
    
    for dataId in dicomVol:
        print("Processing data series", dataId)
        
        for image, mask in zip(dicomVol[dataId], maskVol[dataId]):
            image.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian
            shape_image = image.pixel_array.shape
            
            # Convert to float to avoid overflow or underflow losses.
            image_image_2d = image.pixel_array.astype(float)

            # Rescaling grey scale between 0-255
            image_2d_scaled = (np.maximum(image_image_2d,0) / image_image_2d.max()) * 255.0

            # Convert to uint
            image_2d_scaled = np.uint8(image_2d_scaled)

            image_output_name = trainingImagePath + "/" + format(slice_count, '05d') + ".png"
            # Write the PNG file
            img = Image.fromarray(image_2d_scaled, 'L')
            img.save(image_output_name)

            shape_mask = mask.shape
            img = Image.fromarray(mask)
            mask_output_name = trainingMaskPath + "/" + format(slice_count, '05d') + ".png"
            img.save(mask_output_name)

            slice_count += 1

In [6]:
def createTestingData(sortingDirection:int, imageDicomPath:str="./Dataset/testing"):
    print("Creating testing PNG files.")
    testingImagePath = "./tmp/testingImage"
    
    try:
        os.mkdir(testingImagePath)
    except OSError:
        print ("Creation of the directory %s failed" % testingImagePath)
    else:
        print ("Successfully created the directory %s " % testingImagePath)
    
    image_dcms = [pydicom.read_file(imageDicomPath + '/' + f, force=True) \
                  for f in listdir(imageDicomPath) if isfile(join(imageDicomPath,f)) if f.endswith(".dcm")]
    image_dcms.sort(key = lambda x: int(x[0x20, 0x32][sortingDirection]))
    
    dcm_count = 0

    for image in image_dcms:

        # In case the dicom file does not contain certain meta tags
        image.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

        shape_image = image.pixel_array.shape

        # Convert to float to avoid overflow or underflow losses.
        image_image_2d = image.pixel_array.astype(float)

        # Rescaling grey scale between 0-255
        image_2d_scaled = (np.maximum(image_image_2d,0) / image_image_2d.max()) * 255.0

        # Convert to uint
        image_2d_scaled = np.uint8(image_2d_scaled)
        
        image_output_name = testingImagePath + "/" + str(dcm_count) + ".png"

        # Write the PNG file
        with open(image_output_name, 'wb') as png_file:
            w = png.Writer(shape_image[1], shape_image[0], greyscale=True)
            w.write(png_file, image_2d_scaled)

        dcm_count += 1

    print ("\n Done! Converted "+ str(dcm_count) + " images.")

In [7]:
dicomPaths, maskPaths = getDataPaths()

In [8]:
createTrainingData(dicomPaths, maskPaths, 1)

Creating training PNG files.
Successfully created the directory ./tmp 
Successfully created the directory ./tmp/trainingImage 
Successfully created the directory ./tmp/trainingMask 
Processing data series 0
Processing data series 1
Processing data series 2
Processing data series 3


In [ ]:
trainUnet(COLOR_DICT, multiLabel=True)

W0925 11:54:30.739212 26320 deprecation.py:506] From C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\tensorflow\python\ops\init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11101714907857881160
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6667611669
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4515934453984990330
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


W0925 11:54:31.884213 26320 deprecation.py:323] From C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 256, 256, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         leaky_re_lu[0][0]                
______________________________________________________________________________________________

Epoch 1/5
4096/4096 [==============================] - 1625s 397ms/step - loss: 2.3588 - acc: 0.9169
Epoch 2/5
2333/4096 [================>.............] - ETA: 11:49 - loss: 0.2529 - acc: 0.9825

In [21]:
createTestingData(1)

Creating testing PNG files.
Creation of the directory ./tmp/testingImage failed

 Done! Converted 112 images.


In [22]:
testUnet(COLOR_DICT, "./model_archive/segmentation_09_24_15_46.hdf5", multiLabel=True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14696351576809339985
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6667611669
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9012529607453863858
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_48 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_20 (LeakyReLU

112/112 [==============================] - 4s 34ms/step
Successfully created the directory ./predicted_segmnetation_09_24_17_52 


C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\0.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:44.842897 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\1.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:44.901897 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\2.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:44.960889 14420 util.py:64] 

W0924 17:52:47.289897 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0924 17:52:47.355895 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0924 17:52:47.420889 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
W0924 17:52:47.486861 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\41.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:47.557862 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this

C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\77.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:49.941897 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\78.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:50.001888 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\79.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:50.061859 14420 util.py:6

C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\99.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:51.417859 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\100.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:51.481858 14420 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\Zifeng Li\Anaconda3\envs\Helmsley3DMIP\lib\site-packages\skimage\io\_io.py:141: UserWarning: ./predicted_segmnetation_09_24_17_52\101.png is a low contrast image
  warn('%s is a low contrast image' % fname)
W0924 17:52:51.542858 14420 util.py